<a href="https://colab.research.google.com/github/yuanjinren/DS-Unit-4-Sprint-1-NLP/blob/main/module4-topic-modeling/YuanjinRen_LS_DS_414_Topic_Modeling_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [36]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

# from gensim.models.ldamulticore import LdaMulticore
from gensim import models

import pandas as pd

In [15]:
amazon = pd.read_csv('/content/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [16]:
amazon.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",AmazonBasics,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [18]:
amazon['reviews.text'][3]

'Seem to work as well as name brand batteries at a much better price'

In [5]:
import spacy.cli
spacy.cli.download('en_core_web_lg')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [6]:
nlp = spacy.load('en_core_web_lg')

In [19]:
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)
stopwords = nlp.Defaults.stop_words.union([',', '-'])

In [12]:
def get_lemmas(x):
    lemmas = []
    for token in nlp(x):
        if (token.is_stop!=True) and (token.is_punct!=True):
            lemmas.append(token.lemma_)
    return lemmas

In [21]:
from tqdm import tqdm
tqdm.pandas()
amazon['lemmas'] = amazon['reviews.text'].progress_apply(get_lemmas)

100%|██████████| 28332/28332 [06:04<00:00, 77.65it/s]


In [ ]:
# STOPWORDS = set(STOPWORDS).union(set(['batteries', 'battery', 'I', 'amazon', 'i', 'Amazon', 'it', "it's", 'it.', 'the', 'this']))

# def tokenize(text):
#     return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [ ]:
# amazon['tokens'] = amazon['reviews.text'].apply(tokenize)

In [23]:
amazon[['reviews.text', 'lemmas']][:10]

,reviews.text,lemmas
0,I order 3 of them and one of the item is bad q...,"[order, 3, item, bad, quality, miss, backup, s..."
1,Bulk is always the less expensive way to go fo...,"[bulk, expensive, way, product, like]"
2,Well they are not Duracell but for the price i...,"[Duracell, price, happy]"
3,Seem to work as well as name brand batteries a...,"[work, brand, battery, well, price]"
4,These batteries are very long lasting the pric...,"[battery, long, last, price, great]"
5,Bought a lot of batteries for Christmas and th...,"[buy, lot, battery, Christmas, AmazonBasics, C..."
6,ive not had any problame with these batteries ...,"[have, problame, battery, order, past, pleased]"
7,Well if you are looking for cheap non-recharge...,"[look, cheap, non, rechargeable, battery, perf..."
8,These do not hold the amount of high power jui...,"[hold, high, power, juice, like, energizer, du..."
9,AmazonBasics AA AAA batteries have done well b...,"[AmazonBasics, AA, AAA, battery, appear, good,..."


In [24]:
id2word = corpora.Dictionary(amazon['lemmas'])

In [25]:
corpus = [id2word.doc2bow(text) for text in amazon['lemmas']]

In [30]:
id2word[20]

'brand'

In [27]:
corpus[5]

[(4, 2),
 (20, 2),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 2),
 (38, 1),
 (39, 1)]

In [28]:
# Human readable format of corpus (term-frequency)
[(id2word[word_id], word_count) for word_id, word_count in corpus[5]]

[('battery', 2),
 ('brand', 2),
 ('Amazon', 1),
 ('AmazonBasics', 1),
 ('Basic', 1),
 ('Cell', 1),
 ('Christmas', 1),
 ('arrive', 1),
 ('buy', 2),
 ('difference', 1),
 ('easy', 1),
 ('good', 1),
 ('hand', 1),
 ('house', 1),
 ('lot', 2),
 ('notice', 1),
 ('purchase', 1)]

In [31]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True)


In [32]:
from pprint import pprint

In [33]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.530*"great" + 0.120*"price" + 0.024*"hold" + 0.017*"Buy" + '
  '0.012*"graphic" + 0.010*"functionality" + 0.010*"buy" + 0.009*"wrong" + '
  '0.008*"point" + 0.006*"complaint"'),
 (1,
  '0.097*"tablet" + 0.084*"use" + 0.048*"old" + 0.041*"kid" + 0.034*"year" + '
  '0.032*"buy" + 0.032*"get" + 0.030*"fire" + 0.030*"kindle" + 0.028*"Fire"'),
 (2,
  '0.054*"app" + 0.040*"8" + 0.034*"download" + 0.032*"look" + 0.021*"web" + '
  '0.021*"reading" + 0.019*"free" + 0.019*"take" + 0.018*"user" + '
  '0.016*"memory"'),
 (3,
  '0.248*"good" + 0.105*"game" + 0.068*"nice" + 0.063*"perfect" + '
  '0.048*"price" + 0.031*"case" + 0.023*"value" + 0.021*"add" + '
  '0.020*"friendly" + 0.016*"well"'),
 (4,
  '0.114*"easy" + 0.075*"purchase" + 0.071*"book" + 0.049*"need" + '
  '0.048*"Kindle" + 0.029*"like" + 0.028*"thing" + 0.026*"happy" + 0.025*"set" '
  '+ 0.023*"light"'),
 (5,
  '0.046*"size" + 0.040*"device" + 0.038*"new" + 0.031*"Amazon" + '
  '0.031*"battery" + 0.029*"video" + 0.028*"littl

In [38]:
from gensim.models.coherencemodel import CoherenceModel

In [39]:
# Compute Perplexity
print ('\nPerplexity: ' , lda_model.log_perplexity(corpus))

# Compute the Coherence Score
coherence_model_lda = CoherenceModel(model = lda_model,
                                     texts = amazon['lemmas'],
                                     dictionary=id2word,
                                     coherence= 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score:', coherence_lda)


Perplexity:  -7.562061355519513

Coherence Score: 0.4271492585831475


In [59]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.0MB/s 
     |████████████████████████████████| 552kB 12.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=ce8f3411c0e0e3dee9ebe243a0b6f748820b18c61803ebcfc4621425ab6794af
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=4d9441f22dbe22e77ac451265e46a5b110e40860a5236f2296737c2568d3f0a3
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [60]:
import pyLDAvis.gensim

In [61]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.236762  0.183245       1        1  18.934645
5      0.083001 -0.040373       2        1  13.617219
4      0.007440 -0.217229       3        1  10.988585
8     -0.172571  0.125036       4        1   9.491775
9     -0.049203  0.028677       5        1   8.872351
2      0.069608 -0.140952       6        1   8.489617
7      0.275454  0.131234       7        1   8.407063
6     -0.015066 -0.280806       8        1   8.090824
3     -0.218443  0.122111       9        1   7.613894
0     -0.216984  0.089056      10        1   5.494033, topic_info=              Term         Freq        Total Category  logprob  loglift
22           great  9600.000000  9600.000000  Default  30.0000  30.0000
88            love  6704.000000  6704.000000  Default  29.0000  29.0000
34            good  6214.000000  6214.000000  Default  28.0000  28.0000
3961        tablet  9891.000000  9891.000000  Default  27.0000  27.0000
11            work  4657.000000  4657.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
7245  intermediate    28.440432    29.251741  Topic10  -6.4567   2.8734
19           price  2173.937744  4840.353516  Topic10  -2.1202   2.1011
738          point   149.919250   275.226318  Topic10  -4.7944   2.2940
692      complaint   116.074760   252.400970  Topic10  -5.0503   2.1247
31             buy   176.435822  5246.678223  Topic10  -4.6316  -0.4909

[379 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
5730      8  0.999257      $
210       2  0.998135     10
213       3  0.998375      2
2003      7  0.993819    2nd
0         1  0.996994      3
...     ...       ...    ...
11        4  0.999729   work
606       9  0.993749  worry
246       1  0.998305  worth
695      10  0.996010  wrong
101       1  0.999412   year

[367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 5, 9, 10, 3, 8, 7, 4, 1])

1. Based on above visualization, there are 10 topics, sizes of all topics are very similar. 
2. 10 topics do not spread evenly on the plane. Topic 4, 9 and 10 are clustered together, and have overlaps partially . 
3. Topic 1 and 7 have overlaps. And topic 3 and 8 also have overlaps. 
4. Take topic 5 as an example, the top-30 most relevant terms include 'love, gift, granddaughter ...' It seems that this topic is associated with giving presents to relatives. 
5. Topic 6 has words like 'app, google, memory, download..' It seems that this topic is related to technology. 

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling